In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
import pandas as pd
from nltk.corpus import stopwords
from textblob import Word, TextBlob
from collections import Counter
import six
import textblob
import gensim
import pickle

[nltk_data] Downloading package stopwords to /home/r1/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/r1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/r1/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/r1/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
def get_lemma_textblob(raw_text):
    """
    Get lemma from text
    :param raw_text: Raw text to get lemma from
    :type raw_text: str
    :rtype: list of lemmas
    """
    blob = TextBlob(raw_text)
    return [Word(tag[0], tag[1]).lemma.lower() for tag in blob.pos_tags]

def filter_corpus(text_corpus, words_to_keep = [], words_to_filter = [], n_most_common=0, 
                  rare_words_threshold=10, replace_rare = False):
    """
    Filter out most common, rare, and user supplied words
    :param corpus: a list of list of lemmas as a corpus
    :param words_to_filter: a set of words to filter
    :param n_most_common: top n most frequent words to remove
    :param rare_words_threshold: filter out words if frequency is less than the threshold
    :param replace_rare: if true replace rare words using "UNK", else remove rare words
    :return: a list of list of lemmas as a corpus
    """
    print("Start filtering")
    # remove any token that contains digit
    text_corpus = [[word for word in sub_list if not any(c.isdigit() for c in word)]for sub_list in text_corpus]
    all_tokens = [item for sub_list in text_corpus for item in sub_list]  # flatten the corpus list
    
    c = Counter(all_tokens)
    common_words = [pair[0] for pair in c.most_common(n_most_common) if pair[0] not in words_to_keep] + [""]
    #print("The most %d common words:"%(n_most_common))
    #print(common_words)
    #print("Filtered stop words:")
    #print(words_to_filter)
    if replace_rare is True:
        filtered = set(common_words + list(words_to_filter))
        rare_words = set([k for k, v in six.iteritems(c) if v <= rare_words_threshold and k not in words_to_keep])
        text_corpus = [['UNK' if word in rare_words else word for word in text] for text in text_corpus]
    else:
        rare_words = [k for k, v in six.iteritems(c) if v <= rare_words_threshold and k not in words_to_keep]
        filtered = set(common_words + rare_words + list(words_to_filter))
    #print("Rare words:")
    #print(rare_words)
    texts = [[str(word) for word in text if word not in filtered] for text in text_corpus]
    print("Done filtering")
    return texts

def text_processing(additional_stop_words = [], words_to_keep = [], n_most_common=100, 
                    rare_words_threshold=100, replace_rare=False, gram='unigram', bigram=10):
    """
    - load data
    - lemmatization
    - remove common words and rare words
    - get n_gram tokens
    :param additional_stop_words: tailor-made words to remove for the corpus
    :type additional_stoop_words: list of str
    :param n_most_common,rare_words_threshold,replace_rare: same with function filter_corpus()
    :param gram: 'unigram', 'bigram', 'trigram', phrases length using gensim
    :type gram: str
    :param bigram: threshold for genism phrase
    :type bigram: int
    :return: clean tokenized documents
    :rtype: list of list
    """
#     print('Loading raw text data')
#     sec_10k = pd.read_pickle('data/10k_raw.pickle')
#     #raw = sec_10k.head().copy()
#     #raw_documents = raw['mda_text'].tolist()
#     raw_documents = sec_10k['mda_text'].tolist()
#     #raw_documents = raw_documents[0:10]
#     print('Lemmatizating')
#     lemma_documents = [get_lemma_textblob(document) for document in raw_documents]
#     pickle.dump(lemma_documents, file=open("data/lemma_documents.pickle", 'wb'))
    print('Loading lemmatized documents')
    lemma_documents = pickle.load(file=open("data/10k_lemmatized.pickle", 'rb'))
    print('Cleaninng')
    stop_words = stopwords.words('english') + additional_stop_words
    clean_documents = filter_corpus(lemma_documents, words_to_keep, stop_words, n_most_common, rare_words_threshold, 
                                   replace_rare)
    print('N_gram')
    if gram == 'unigram':
        return clean_documents
    if gram == 'bigram':
        bigram_transformer = gensim.models.Phrases(clean_documents, min_count=1, threshold=bigram)
        bigram = list(bigram_transformer[clean_documents])
        return bigram
    if gram == 'trigram':
        bigram_transformer = gensim.models.Phrases(clean_documents, min_count=1)
        bigram = list(bigram_transformer[clean_documents])
        trigram_transformer = gensim.models.Phrases(bigram, min_count=1)
        trigram = list(trigram_transformer[bigram])
        return trigram

def similar_words(word2vec_model, dimension, n=5):
    """
    given a dimension of seed word or list of seed words find most similar words in word2vec corpus, based on cosine similarity
    :param word2vec_model: gensim word2vec model
    :type dimension: str
    :param n: the most n similar words
    :type n: int
    :rtype: list of (similar_word, similarity) tuples
    """
    similar_words = []
    for word in seed_words[dimension]:
        if isinstance(word, list):
            #make sure every word in seed word list is in word2vec corpus
            updated_word = [item for item in word if item in word2vec_model.vocab]
            try:
                for pair in word2vec_model.most_similar(updated_word, topn=n):
                    similar_words.append(pair)
            except:
                pass
                #similar_words[', '.join(word)] = ('All the words in this seed word list not found in corpus', 0)
        else:
            try:
                for pair in word2vec_model.similar_by_word(word, topn=n):
                    similar_words.append(pair)
            except:
                pass
                #similar_words[word] = ('Seed word not found in corpus', 0)
    return similar_words

def expand_seed_words(documents, fname, min_count=1, size=100, window=5, workers=16):
    print('Building word2vec model')
    model = gensim.models.Word2Vec(documents, min_count=min_count, size=size, window=window, workers=workers)
    model.save(fname)
    model.init_sims(replace=True)
    expanded_words = {}
    for dimension in seed_words:
        expanded_words[dimension] = similar_words(word2vec_model=model, dimension=dimension)
    return expanded_words

def output(expanded_words_list, seed_words, n, fname):
    combined_list = {}
    for dimension in seed_words:
        combined_list[dimension] = []
    for item in expanded_words_list:
        for k,v in item.items():
            for expanded_word_tuple in v:
                current_expanded = [tup[0]for tup in combined_list[k]]
                if expanded_word_tuple[0] not in current_expanded:
                    combined_list[k].append(expanded_word_tuple)
    with open(fname, 'w') as text_file:
        for k in combined_list:
            combined_list[k].sort(key=lambda tup: tup[1], reverse=True)
            combined_list[k] = [tup[0] for tup in combined_list[k]]
            if len(combined_list[k]) >= n:
                combined_list[k] = combined_list[k][0:n]
                print("The dimension is: {} {} words".format(k, n), file=text_file)
                print("===========================================================================", file=text_file)
                print(combined_list[k], file=text_file)
                print('\n', file=text_file)
            else:
                print("The dimension is: {}. Less than {}words, output {}.".format(k, n, len(combined_list[k])), file=text_file)
                print("===========================================================================", file=text_file)
                print(combined_list[k], file=text_file)
                print('\n', file=text_file)
    return combined_list


### construct original seed words based on six culture dimensions, for every dimension, only choose words with positive loadings greater than 0.4 and update using self judgement

In [3]:
seed_words_oreilly = {"adaptability":
["being_innovative", "be_innovative", "risk_taking", "risk_taken","take_risk", "being_willing_to_experiment", "fast_moving",
"being_quick_to_take_advantage_of_opportunities", "not_being_constrained_by_many_rules", "adaptability"],
"integrity":
["having_integrity", 'have_integrity', 'be_honest', "having_high_ethical_standards", "being_honest", "respecting_individuals",
"being_fair", 'be_fair', 'be_supportive'],
"collaborative":["working_in_collaboration_with_others", "being_team_oriented", "cooperative", "being_supportive",
"avoiding_conflict", 'be_supportive', 'avoid_conflict'],
"results_oriented":['results_oriented', "being_results_oriented", "having_high_expectations_for_performance", "achievement_oriented"],
"customer_oriented":['customer_oriented', "being_customer_oriented", "listening_to_customers", "being_market_driven", 'market_driven'],
"detail_oriented":["paying_attention_to_detail", "emphasizing_quality", 'emphasize_quality', "being_precise", 'be_precise', 'detail', 'precise', 
                  'detail_oriented']}

In [4]:
seed_words = {"integrity":["integrity", "ethics", "accountability", "trust", "honesty", 
                           "responsibility", "fairness", "transparency", 
                           "ownership", "fair", "honest", 
                           "ethical", "transparent", 
                          ['integrity', 'ethics', 'accountability', 'honesty', 'honest', 'ethical', 'trust'], 
                          ['transparency', 'transparent', 'fair', 'fairness']],
"teamwork":["teamwork", "collaboration", "cooperation", "collaborative", "cooperative", 
           ['teamwork', 'collaboration', 'collaborative', 'cooperative']],
"innovation":["innovation", "creativity", "excellence", "improvement", "passion", 
              "pride", "leadership", "growth", "performance", "efficiency", 
              "efficient", "results", "result", 'innovative', 'creative', 'fast_moving', 'move_fast', 'result_oriented', 
              ['innovation', 'innovative', 'creativity', 'creative'], 
             ['leadership', 'leader'], 
             ['efficiency', 'efficient', 'results', 'result', 'performance']],
"respect":["respect", "diversity", "inclusion", "development", 
           "talent", "employees", "employee", 
           "dignity", "empowerment", 'respect_individual', 
          ['talent', 'employees', 'employee', 'development'], 
          ['diversity', 'respect', 'inclusion', 'empowerment']],
"quality":["quality", "customer", "meet_needs", 'meet_need', 
           "commitment", "make_a_difference", "dedication", 'detail_oriented', 
           "value", "exceed_expectations", "exceed_expectation", 'emphasize_quality', 
          ['quality', 'customer'], 
          ['commitment', 'dedication', 'value']],
"safety":["safety", "health", "healthy", "work_life_balance", "flexibility", 
         ['safety', 'health', 'healthy']],
"community":["community", "environment", "caring", "citizenship", 
            ['community', 'environment', 'caring', 'citizenship']],
"communication":["communication", "openness", "open_mind", 'communicate', 
                ['communication', 'openness', 'communicate']],
"hard_work":["hard_work", "reward", "fun", "energy", ['reward', 'fun', 'energy']]}

bigram = {"bigrams":['being_innovative', 'be_innovative', 'risk_taking', 'risk_taken', 'take_risk', 'fast_moving', 'move_fast', 
          'have_integrity', 'having_integrity', 'be_honest', 'being_honest', 'respecting_individuals', 'respecting_individual', 
          'respect_individuals', 'respect_individual', 'being_fair', 'be_fair', 'be_supportive', 'avoiding_conflict', 
          'avoid_conflict', 'results_oriented', 'result_oriented', 'achievement_oriented', 'customer_oriented', 
          'market_driven', 'detail_oriented', 'emphasizing_quality', 'emphasize_quality', 'being_precise', 
          'be_precise']}


In [5]:
words_to_keep = ['empowerment', 'customer', 'value', 'honesty', 'honest', 'result', 'caring', 'openness']
words_to_remove = ['rx', 'chmp', 'spse', '-results', 'cia', 'openness', 'and/or', 'apb', 'cpt', 'qt', 'cte', 'mkg', 'nhs', 
                  '1350', 'rep', 'iplex', 'hap-tm-', 'hap']

In [6]:
# before adding words_to_keep result
# documents = text_processing()
# expanded_words = expand_seed_words(documents=documents)
# for k, v in expanded_words.items():
#     print('Culture dimension: %s' %(k))
#     print('=========================================================================================================')
#     print(v)

In [7]:
%%time
# after adding words_to_keep unigram result
documents = text_processing(additional_stop_words=words_to_remove, words_to_keep=words_to_keep)
expanded_words_unigram = expand_seed_words(documents=documents, fname='data/10k_unigram')

pickle.dump(expanded_words_unigram, file=open("data/10k_expanded_unigram.pickle", 'wb'))
del documents

Loading lemmatized documents
Cleaninng
Start filtering
Done filtering
N_gram
Building word2vec model
CPU times: user 3h 44min 34s, sys: 3min 20s, total: 3h 47min 55s
Wall time: 47min 2s


In [8]:
%%time
# after adding words_to_keep bigram result phrase threshold=10
documents = text_processing(additional_stop_words=words_to_remove, words_to_keep=words_to_keep, n_most_common=100, 
                            rare_words_threshold=100, replace_rare=False, gram='bigram', bigram=10)
expanded_words_bigram_10 = expand_seed_words(documents=documents, fname='data/10k_bigram_10')

pickle.dump(expanded_words_bigram_10, file=open("data/10k_expanded_bigram_10.pickle", 'wb'))
del documents

Loading lemmatized documents
Cleaninng
Start filtering
Done filtering
N_gram


/opt/anaconda3/lib/python3.5/site-packages/gensim/models/phrases.py:248: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


Building word2vec model
CPU times: user 4h 9min 9s, sys: 6min 3s, total: 4h 15min 12s
Wall time: 1h 26min 52s


In [9]:
%%time
# after adding words_to_keep bigram result phrase threshold=1
documents = text_processing(additional_stop_words=words_to_remove, words_to_keep=words_to_keep, n_most_common=100, 
                    rare_words_threshold=100, replace_rare=False, gram='bigram', bigram=1)
expanded_words_bigram_1 = expand_seed_words(documents=documents, fname='data/10k_bigram_1')

pickle.dump(expanded_words_bigram_1, file=open("data/10k_expanded_bigram_1.pickle", 'wb'))
del documents

Loading lemmatized documents
Cleaninng
Start filtering
Done filtering
N_gram


/opt/anaconda3/lib/python3.5/site-packages/gensim/models/phrases.py:248: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


Building word2vec model
CPU times: user 3h 45min 32s, sys: 6min 55s, total: 3h 52min 27s
Wall time: 1h 24min 38s


In [10]:
expanded_unigram_10k = pickle.load(file=open("data/10k_expanded_unigram.pickle", 'rb'))
expanded_bigram_10_10k = pickle.load(file=open("data/10k_expanded_bigram_10.pickle", 'rb'))
expanded_bigram_1_10k = pickle.load(file=open("data/10k_expanded_bigram_1.pickle", 'rb'))
#expanded_unigram_glassdoor = pickle.load(file=open("data/glassdoor_expanded_unigram.pickle", 'rb'))
#expanded_bigram_10_glassdoor = pickle.load(file=open("data/glassdoor_expanded_bigram_10.pickle", 'rb'))
#expanded_bigram_1_glassdoor = pickle.load(file=open("data/glassdoor_expanded_bigram_1.pickle", 'rb'))

In [11]:
expanded_words_list = [expanded_unigram_10k, expanded_bigram_10_10k, expanded_bigram_1_10k]
result = output(expanded_words_list, seed_words, 50, 'data/10k_result_50.txt')

In [32]:
# final_expanded_words = {}
# for dimension in seed_words:
#     final_expanded_words[dimension] = {}
# for expanded_words in expanded_words_list:
#     for dimension in expanded_words:
#         for one_expanded_word in expanded_words[dimension]:
#             for k,v in one_expanded_word.items():
#                 if isinstance(v, str):
#                     v = [v]
#                 if k in final_expanded_words[dimension]:
#                     final_expanded_words[dimension][k] = set(list(final_expanded_words[dimension][k]) + v)
#                 else:
#                     final_expanded_words[dimension][k] = v